In [1]:
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from PIL import Image
import numpy as np

In [12]:
with open(r"C:\Users\Raum\Desktop\array_of_frame\2023-09-07_14-56-57_Video_array.npy", 'rb') as f:
    a = np.load(f)
a.shape

(1967,)

In [10]:
# for i,j in enumerate(a):
#     print(i,j)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1
44 1
45 1
46 1
47 1
48 1
49 1
50 1
51 1
52 1
53 1
54 1
55 1
56 1
57 1
58 1
59 1
60 1
61 1
62 1
63 1
64 1
65 1
66 1
67 1
68 1
69 1
70 1
71 1
72 1
73 1
74 1
75 1
76 1
77 1
78 0
79 0
80 0
81 0
82 0
83 0
84 1
85 1
86 1
87 1
88 1
89 1
90 1
91 1
92 1
93 1
94 1
95 1
96 0
97 0
98 0
99 0
100 0
101 1
102 0
103 1
104 0
105 1
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 1
135 1
136 1
137 1
138 1
139 1
140 1
141 1
142 1
143 1
144 1
145 1
146 1
147 1
148 1
149 1
150 1
151 1
152 1
153 1
154 1
155 1
156 1
157 1
158 1
159 1
160 1
161 1
162 1
163 1
164 1
165 1
166 1
167 1
168 1
169 1
170 1
171 1
172 1
173 1
174 1
175 1
176 1
177 1
178 1
179 1
180 1
181 1
182 1
183 1
184 1


In [3]:
embedder = FaceNet()
detector = MTCNN()

In [4]:
def extraction_keras_lock(*file_name,show=None):
  if type(file_name) is tuple:
    file_name = [*file_name,]
  result = []
  for i in file_name:
    image = Image.open(i).convert('RGB')
    im_arr = np.array(image)
    N=0
    faces = detector.detect_faces(im_arr)
    x1,y1,width,height = faces[N]['box'] # faces[N]['box']
    x2,y2 = (x1+width),(y1+height)
    face = im_arr[y1:y2,x1:x2] # ตัดเอาเฉพาะใบหน้า
    image_face = Image.fromarray(face)
    image_face= image_face.resize((224,224))
    face_img = np.expand_dims(image_face,axis=0)
    result.append(embedder.embeddings(face_img))

  return result

In [5]:
def extraction_keras_all_cv(im_arr):
    position_crop = []
    embedding_faces = []
    position = []
    faces = detector.detect_faces(im_arr)
    
    if faces == []:
        return [],[]

    for i in faces:
        x1,y1,width,height = i['box']
        x2,y2 = (x1+width),(y1+height)
        position_crop.append([y1,y2,x1,x2])
        position.append([x1,y1,x2,y2])

    # print(len(position_crop))
    for i in position_crop:
        # print(i)
        face = im_arr[i[0]:i[1],i[2]:i[3]]
        image_face = cv2.resize(face,(224,224))
        faces_crops = np.expand_dims(image_face,axis=0)
        embedding_faces.append(embedder.embeddings(faces_crops))
        # print(embedding_faces)
    return embedding_faces,position

In [6]:
image=cv2.imread(r'C:\Users\Raum\Desktop\jec\code\dataface\pitha.jpg')
im_arr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
embedding_faces,position = extraction_keras_all_cv(im_arr)
xx = extraction_keras_lock(r'C:\Users\Raum\Desktop\jec\code\dataface\Pita.jpg',r'C:\Users\Raum\Desktop\jec\code\dataface\siri.jpg')

In [21]:

log_cosine_similarity = []

Detection_check = np.zeros((len(position)))
for j in range(len(xx)):

    for i in range(len(position)):
        similar = cosine_similarity(xx[j], embedding_faces[i])
        log_cosine_similarity.append(similar)
        if similar>0.55 and Detection_check[i] ==0 :
            Detection_check[i] = 1
            break

for i in range(len(position)):
    if Detection_check[i] == 1:
          pass
    else :
        censor_region = (position[i][0],position[i][1],position[i][2],position[i][3])

        censored_area = image[censor_region[1]:censor_region[3], censor_region[0]:censor_region[2]]

        censored_width, censored_height = censored_area.shape[1], censored_area.shape[0]


        pixel_size = 12
        censored_area = cv2.resize(censored_area, (censored_width // pixel_size, censored_height // pixel_size))
        censored_area = cv2.resize(censored_area, (censored_width, censored_height), interpolation=cv2.INTER_NEAREST)

        image[censor_region[1]:censor_region[3], censor_region[0]:censor_region[2]] = censored_area



[array([[0.7874079]], dtype=float32),
 array([[0.2973188]], dtype=float32),
 array([[-0.03770198]], dtype=float32),
 array([[0.08064354]], dtype=float32),
 array([[0.27171814]], dtype=float32),
 array([[-0.05674545]], dtype=float32),
 array([[0.0596335]], dtype=float32),
 array([[0.05591889]], dtype=float32),
 array([[0.20720986]], dtype=float32),
 array([[0.06897811]], dtype=float32),
 array([[0.7230322]], dtype=float32)]

In [22]:
log_cosine_similarity

[array([[0.7874079]], dtype=float32),
 array([[0.2973188]], dtype=float32),
 array([[-0.03770198]], dtype=float32),
 array([[0.08064354]], dtype=float32),
 array([[0.27171814]], dtype=float32),
 array([[-0.05674545]], dtype=float32),
 array([[0.0596335]], dtype=float32),
 array([[0.05591889]], dtype=float32),
 array([[0.20720986]], dtype=float32),
 array([[0.06897811]], dtype=float32),
 array([[0.7230322]], dtype=float32)]

In [23]:
cv2.imwrite('ggggg.jpg',image)

True

In [2]:
import cv2
import numpy as np
import time
import threading

with open(r"C:\Users\Raum\Desktop\array_of_frame\2023-09-07_14-56-57_Video_array.npy", 'rb') as f:
    a = np.load(f)
print(a.shape)

image_files = []

path_file ="C:/Users/Raum/Desktop/array_of_frame/allframe/"
for i in range(a.shape[0]):
    # print(path_file+str(i)+'.jpg')
    image_files.append(path_file+str(i)+'.jpg')

(1967,)


In [3]:
start = time.time()
stop_event = threading.Event()
def read_image(file_path,index,image_all = [],image_dex=[]):
    image = cv2.imread(file_path)
    # print(f"Image {file_path} dimensions: {image.shape}")
    image_all.append((image,index))
    image_dex.append(index)

threads = []
image_all = []
image_dex = []
for i,file_path in enumerate(image_files):
    thread = threading.Thread(target=read_image, args=(file_path,i,image_all,image_dex))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# for i in range(a.shape[0]):
#     print(path_file+str(i)+'.jpg')
#     frame = cv2.imread(path_file+str(i)+'.jpg')
#     image_all.append((frame,i))

print("All threads have finished.")
stop_event.set()
end = time.time()
print(end - start)

def take_tail(elem):
    return elem[1]

image_all.sort(key=take_tail)

All threads have finished.
7.344156503677368


In [4]:
# print(len(image_all))
# # image_dex
# rgb_img = cv2.cvtColor(image_all[12], cv2.COLOR_BGR2RGB)
# image=Image.fromarray(rgb_img)
# image
# # image_dex
# np.array(len(image_all))

In [9]:
sstrr = time.strftime("%Y-%m-%d_%H-%M-%S")
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output_path = "C:/Users/Raum/Desktop/writed_video/{}_Video_array.mp4".format(sstrr)
new_width = 1920
new_height = 1080
fps = 25
out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

for i,(frame,index) in enumerate(image_all):
    out.write(frame)

out.release()
cv2.destroyAllWindows()


